In [3]:
!pip install pymc


Defaulting to user installation because normal site-packages is not writeable
  Using cached arviz-0.22.0-py3-none-any.whl.metadata (8.9 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pandas-2.3.1-cp312-cp312-win_amd64.whl.metadata (19 kB)
   ---------------------------------------- 0.0/535.5 kB ? eta -:--:--
   ---------------------------------------- 0.0/535.5 kB ? eta -:--:--
   ---------------------------------------- 535.5/535.5 kB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------------------------- -------------- 1.0/1.6 MB 5.0 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 4.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------------------------- -------- 1.3/1.7 MB 6.7 MB/s eta 0:00:01
   -----------------------------

  DEPRECATION: Building 'logical-unification' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'logical-unification'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit-authenticator 0.4.2 requires streamlit>=1.37.0, but you have streamlit 1.35.0 which is incompatible.
torchaudio 2.7.1+cpu requires torch==2.7.1, but you have torch 2.2.1 which is incompatible.
torchvision 0.22.1+cpu requires torch==2.7.1, but you have torch 2

In [2]:
import pymc as pm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import arviz as az

# Load data
prices = pd.read_csv('../data/BrentOilPrices.csv', parse_dates=['Date'])
prices = prices.sort_values('Date')
prices['log_return'] = np.log(prices['Price']) - np.log(prices['Price'].shift(1))
prices = prices.dropna()

print("Data successfully loaded")

C:\Users\dell\AppData\Local\Temp\ipykernel_11856\1062013728.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  prices = pd.read_csv('../data/BrentOilPrices.csv', parse_dates=['Date'])


Data successfully loaded


In [ ]:
# Bayesian Change Point Detection
with pm.Model() as single_change_point:
    # Priors
    tau = pm.DiscreteUniform('tau', lower=0, upper=len(prices)-1)
    mu1 = pm.Normal('mu1', mu=0, sigma=10)
    mu2 = pm.Normal('mu2', mu=0, sigma=10)
    sigma = pm.HalfNormal('sigma', sigma=10)
    
    # Expected value
    mu = pm.math.switch(tau >= np.arange(len(prices)), mu1, mu2)
    
    # Likelihood
    returns_obs = pm.Normal('returns_obs', mu=mu, sigma=sigma, 
                          observed=prices['log_return'].values)
    
    # Inference
    trace = pm.sample(2000, tune=1000, cores=2)

# Diagnostics
az.plot_trace(trace)
plt.show()


Multiprocess sampling (2 chains in 2 jobs)
CompoundStep
>Metropolis: [tau]
>NUTS: [mu1, mu2, sigma]


Output()

In [ ]:
# Results
tau_samples = trace['tau']
tau_mean = int(tau_samples.mean())
change_date = prices['Date'].iloc[tau_mean]

print(f"Most probable change point at index {tau_mean} (Date: {change_date})")


In [ ]:
# Plot posterior distribution of tau
plt.figure(figsize=(10, 4))
plt.hist(tau_samples, bins=50, density=True)
plt.axvline(tau_mean, color='red', linestyle='--')
plt.title('Posterior Distribution of Change Point')
plt.xlabel('Time Index')
plt.ylabel('Probability Density')
plt.show()

In [ ]:
# Compare before/after
mu1_samples = trace['mu1']
mu2_samples = trace['mu2']

plt.figure(figsize=(10, 4))
plt.hist(mu1_samples, bins=50, alpha=0.5, label='Before change')
plt.hist(mu2_samples, bins=50, alpha=0.5, label='After change')
plt.title('Comparison of Mean Returns Before and After Change Point')
plt.xlabel('Mean Log Return')
plt.legend()
plt.show()


In [ ]:
# Probability that mean increased
prob_increase = (mu2_samples > mu1_samples).mean()
print(f"Probability that mean return increased: {prob_increase*100:.1f}%")